## Búsqueda semántica 1

NOTA: Recomendable ejecutarlo en un entorno virtual de python

In [ ]:
!pip install smolagents smolagents[litellm] python-dotenv

In [ ]:
!python.exe -m pip install --upgrade pip

### Cargar la clave de hugging face desde el fichero .env

In [1]:
import pprint

In [ ]:
from dotenv import load_dotenv

load_dotenv()

### Bibliotecas específicas para postgres con pgvector

In [ ]:
!pip install psycopg2-binary numpy

### Embeddings

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Cargar modelo y tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_embedding = AutoModel.from_pretrained(model_name)

In [4]:
def get_embedding(text):
    # Tokenizar el texto
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model_embedding(**inputs)
    # Obtener la representación del [CLS] token
    embedding = outputs.last_hidden_state[:, 0, :]
    return embedding.numpy()

In [5]:
def search_similar(query, limit=3):
    query_vector = get_embedding(query)
    result = db.search_similar(query_vector[0], limit)

    print(f'Pregunta: {query}')
    pprint.pprint(result)

### Uso de postgres_vector.py

In [6]:
from postgres_vector import PostgresVector

db = PostgresVector()

In [ ]:
db.connect()

db.create_vector_extension()
db.create_items_table(dimension=384)  # Ajustar según la dimensión de tus embeddings

In [ ]:
name = 'Martillo'
descripcion = 'Martillo herramienta para golpear y clavar'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Destornillador'
descripcion = 'Destornillador herramienta que sirve para atornillar y desatornillar'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Alicates'
descripcion = 'Alicates herramienta para cualquier uso'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Llave inglesa'
descripcion = 'Llave inglesa herramienta para apretar y desapretar'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Tijeras'
descripcion = 'Tijeras herramienta para cortar'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Clavo'
descripcion = 'Clavo que sirve para clavar objetos'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Tornillo'
descripcion = 'Tornillo que sirve para sujetar objetos'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Tuerca'
descripcion = 'Tuerca que sirve para sujetar objetos'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])


name = '¿Cómo te llamas?'
descripcion = '¿Cómo te llamas? pregunta para saber el nombre'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = '¿Cuál es tu nombre?'
descripcion = '¿Cuál es tu nombre? pregunta para saber el nombre'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Dime tu nombre'
descripcion = 'Dime tu nombre pregunta para saber el nombre'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = '¿Cómo estás?'
descripcion =  '¿Cómo estás? pregunta para saber si estás bien o no'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Me encuentro bien'
descripcion = 'Me encuentro bien respuesta para decir que estoy bien'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'Estoy enfermo'
descripcion = 'Estoy enfermo respuesta para decir que estoy '
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

name = 'No me duele nada'
descripcion = 'No me duele nada respuesta para decir que estoy bien'
embedding = get_embedding(descripcion)
db.insert_item(name, descripcion, embedding[0])

In [ ]:
'''
# Actualizar un item
db.update_item(item_id=1, description="Martillo de acero inoxidable")

# Eliminar un item
db.delete_item(item_id=2)
'''

### Ejemplos de búsqueda semántica

In [ ]:
search_similar('qué puedo usar para clavar')

In [ ]:
search_similar('herramientas', 5)

In [ ]:
search_similar('cómo preguntar el nombre', 3)

In [ ]:
db.delete_all_items()

In [21]:
db.disconnect()